In [1]:
import matplotlib.font_manager
import numpy as np
import quocslib 

import matplotlib.pyplot as plt
import matplotlib
matplotlib.font_manager.fontManager.addfont("fonts/ipag.ttf")
matplotlib.rcParams['font.family'] = 'IPAGothic'

In [1]:
import matplotlib.font_manager
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time
from quocslib.utils.AbstractFoM import AbstractFoM
from quocslib.Optimizer import Optimizer
import qutip as qt

# フォント設定
matplotlib.font_manager.fontManager.addfont("fonts/ipag.ttf")
matplotlib.rcParams['font.family'] = 'IPAGothic'

# 電子スピン系の1量子ビット制御問題のFoMクラス
class SpinQubitControl(AbstractFoM):
    def __init__(self, args_dict=None):
        if args_dict is None:
            args_dict = {}
        
        # 系のパラメータ設定
        self.omega0 = args_dict.setdefault("omega0", 2.0 * np.pi)  # ラーモア周波数
        self.n_slices = args_dict.setdefault("n_slices", 100)
        self.is_maximization = args_dict.setdefault("is_maximization", True)
        
        # ドリフトハミルトニアン（静磁場）
        self.H_drift = 0.5 * self.omega0 * qt.sigmaz()
        
        # 制御ハミルトニアン（Y, Y, Z）
        self.H_controls = [
            qt.sigmay(),  # 第1制御：Y方向
            qt.sigmay(),  # 第2制御：Y方向
            qt.sigmaz()   # 第3制御：Z方向
        ]
        
        # 初期状態と目標状態を設定
        self.rho_0 = qt.basis(2, 0) * qt.basis(2, 0).dag()  # |0⟩⟨0|
        self.rho_target = qt.basis(2, 1) * qt.basis(2, 1).dag()  # |1⟩⟨1|
        
        # プロパゲータを保存するためのメモリ確保
        self.propagators = [np.zeros((2, 2), dtype=complex) for _ in range(self.n_slices)]
        self.propagators_computed = False
        
        # FoM最大化のための係数
        self.FoM_factor = 1.0 if self.is_maximization else -1.0
    
    def get_drift_Hamiltonian(self):
        return self.H_drift
    
    def get_control_Hamiltonians(self):
        return self.H_controls
    
    def get_initial_state(self):
        return self.rho_0
    
    def get_target_state(self):
        return self.rho_target
    
    def get_propagator(self, pulses_list=None, time_grids_list=None, parameters_list=None):
        """パルスに基づいてプロパゲータを計算"""
        if pulses_list is None:
            pulses_list = []
        if time_grids_list is None:
            time_grids_list = []
        if parameters_list is None:
            parameters_list = []
        
        # 時間ステップを計算
        dt = time_grids_list[0][-1] / len(time_grids_list[0])
        
        # 各時間ステップでのプロパゲータを計算
        for i in range(self.n_slices):
            # 現在の時間ステップでのハミルトニアン
            H_t = self.H_drift.copy()
            for j, pulse in enumerate(pulses_list):
                if j < len(self.H_controls):
                    H_t += pulse[i] * self.H_controls[j]
            
            # プロパゲータを計算（時間発展演算子）
            self.propagators[i] = (-1j * H_t * dt).expm().full()
        
        self.propagators_computed = True
        return self.propagators
    
    def get_FoM(self, pulses=None, parameters=None, timegrids=None):
        """評価関数（忠実度）の計算"""
        if pulses is None:
            pulses = []
        if parameters is None:
            parameters = []
        if timegrids is None:
            timegrids = []
        
        # プロパゲータがまだ計算されていなければ計算
        if not self.propagators_computed:
            self.get_propagator(pulses_list=pulses, time_grids_list=timegrids, parameters_list=parameters)
        
        self.propagators_computed = False
        
        # 全体のプロパゲータを計算（時間順序積）
        U_total = np.eye(2, dtype=complex)
        for U in self.propagators:
            U_total = U @ U_total
        
        # 初期状態から最終状態を計算
        rho_0_mat = self.rho_0.full()
        rho_final = U_total @ rho_0_mat @ U_total.conj().T
        
        # 目標状態との忠実度を計算
        rho_target_mat = self.rho_target.full()
        fidelity = np.abs(np.trace(np.sqrt(np.sqrt(rho_final) @ rho_target_mat @ np.sqrt(rho_final)))) ** 2
        
        # 最大化のため正の値で返す
        return {"FoM": self.FoM_factor * fidelity}

# 最適化設定
# GRAPEアルゴリズムを使用した最適化辞書の設定
optimization_dictionary = {
    "optimization_client_name": "SpinQubit_GRAPE"
}

# アルゴリズム設定 - GRAPEを使用
optimization_dictionary["algorithm_settings"] = {
    "algorithm_name": "GRAPE",
    "optimization_direction": "maximization"
}

# パルス設定（Y, Y, Z）
# パルス1: Y方向の制御
pulse_y1 = {
    "pulse_name": "Pulse_Y1",
    "upper_limit": 5.0,
    "lower_limit": -5.0,
    "bins_number": 100,
    "amplitude_variation": 1.0,
    "time_name": "time_1",
    "basis": {
        "basis_name": "PiecewiseBasis"
    }
}

# パルス2: Y方向の制御（別の周波数または位相）
pulse_y2 = {
    "pulse_name": "Pulse_Y2",
    "upper_limit": 5.0,
    "lower_limit": -5.0,
    "bins_number": 100,
    "amplitude_variation": 1.0,
    "time_name": "time_1",
    "basis": {
        "basis_name": "PiecewiseBasis"
    }
}

# パルス3: Z方向の制御
pulse_z = {
    "pulse_name": "Pulse_Z",
    "upper_limit": 10.0,
    "lower_limit": -10.0,
    "bins_number": 100,
    "amplitude_variation": 1.0,
    "time_name": "time_1",
    "basis": {
        "basis_name": "PiecewiseBasis"
    }
}

# 時間設定
time1 = {
    "time_name": "time_1",
    "initial_value": 10.0  # 総制御時間
}

# 辞書にパルスと時間を追加
optimization_dictionary["pulses"] = [pulse_y1, pulse_y2, pulse_z]
optimization_dictionary["parameters"] = []
optimization_dictionary["times"] = [time1]

# 最適化の実行
# FoMオブジェクトを作成
fom_object = SpinQubitControl({"is_maximization": True})

# 最適化オブジェクトを作成
optimization_obj = Optimizer(optimization_dictionary, fom_object)

# 最適化を実行
print("最適化を開始します...")
time_start = time.time()
optimization_obj.execute()
time_end = time.time()
print(f"最適化が完了しました。実行時間: {time_end - time_start:.2f}秒")

# 結果の可視化
results = optimization_obj.get_results()
best_controls = results["best_controls"]
best_fom = results["best_fom"]

plt.figure(figsize=(14, 10))

# Y1パルスのプロット
plt.subplot(3, 1, 1)
plt.plot(np.linspace(0, time1["initial_value"], pulse_y1["bins_number"]), best_controls[0], 'r-')
plt.title('Y1制御パルス')
plt.xlabel('時間 (a.u.)')
plt.ylabel('振幅')
plt.grid(True)

# Y2パルスのプロット
plt.subplot(3, 1, 2)
plt.plot(np.linspace(0, time1["initial_value"], pulse_y2["bins_number"]), best_controls[1], 'g-')
plt.title('Y2制御パルス')
plt.xlabel('時間 (a.u.)')
plt.ylabel('振幅')
plt.grid(True)

# Zパルスのプロット
plt.subplot(3, 1, 3)
plt.plot(np.linspace(0, time1["initial_value"], pulse_z["bins_number"]), best_controls[2], 'b-')
plt.title('Z制御パルス')
plt.xlabel('時間 (a.u.)')
plt.ylabel('振幅')
plt.grid(True)

plt.tight_layout()
plt.suptitle(f'最適制御パルス (最終忠実度: {best_fom:.6f})', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

# 制御の時間発展をシミュレート（オプション）
def simulate_dynamics(best_controls, time_total, n_steps):
    # 時間グリッド
    times = np.linspace(0, time_total, n_steps)
    dt = time_total / n_steps
    
    # 量子状態の初期化
    psi = qt.basis(2, 0)  # |0⟩ 状態
    
    # 各時間ステップでの状態を保存
    states = [psi]
    
    # 時間発展
    for i in range(n_steps):
        # 現在のハミルトニアン
        H_t = 0.5 * fom_object.omega0 * qt.sigmaz() + \
              best_controls[0][i] * qt.sigmay() + \
              best_controls[1][i] * qt.sigmay() + \
              best_controls[2][i] * qt.sigmaz()
        
        # 時間発展演算子
        U = (-1j * H_t * dt).expm()
        
        # 状態を更新
        psi = U * psi
        states.append(psi)
    
    return times, states

times, states = simulate_dynamics(best_controls, time1["initial_value"], pulse_y1["bins_number"])

# 確率の計算と可視化
p0 = [qt.fidelity(state, qt.basis(2, 0))**2 for state in states]
p1 = [qt.fidelity(state, qt.basis(2, 1))**2 for state in states]

plt.figure(figsize=(10, 6))
plt.plot(times, p0, 'b-', label='|0⟩状態の確率')
plt.plot(times, p1, 'r-', label='|1⟩状態の確率')
plt.xlabel('時間 (a.u.)')
plt.ylabel('確率')
plt.title('量子状態の時間発展')
plt.legend()
plt.grid(True)
plt.show()

INFO     oc_logger    The optimization direction is maximization
INFO     oc_logger    Random number generator from the numpy library, version 2.2.5
最適化を開始します...
INFO     oc_logger    QuOCS version number: 0.0.66


TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Qobj'